### Transformation

Now we need to go into linear algebra and matrices. If you have not done any matrix math before, I will try to
keep this simple. In linear algebra we have 1-dimensional arrays, like a point. We have written the point class
with X, Y, Z member variables. But, it could have been written as an array of 3 values with X, Y, and Z indices of
0, 1, and 2 respectively.

This prepares us to talk about a transform. The idea is that there is a transformation matrix that maps from one
coordinate system (i.e. robot coordinates) to another coordinate system (i.e. field coordinates). The transformation
is a 4x4 matrix, and the transformation (`xfm`) from coordinate system to another coordinate system written as:
```
     [ xfm[0][0] xfm[0][1] xfm[0][2] xfm[0][3] ] [ X ]   [ Xt ]
     | xfm[1][0] xfm[1][1] xfm[1][2] xfm[1][3] | | Y | = | Yt |
     | xfm[2][0] xfm[2][1] xfm[2][2] xfm[2][3] | | Z |   | Zt |
     [ xfm[3][0] xfm[3][1] xfm[3][2] xfm[3][3] ] [ 1 ]   [ 1  ]
```

If you have not done matrix math before, the multiplication is that you walk horizontally across each row of the
matrix and multiply the column value by the corresponding row value of the point, then sum the result.
 Confused? - OK lets walk through that:
```
    Xt = (xfm[0][0] * X) + (xfm[0][1] * Y) + (xfm[0][2] * Z) + (xfm[0][3] * 1)
    Yt = (xfm[1][0] * X) + (xfm[1][1] * Y) + (xfm[1][2] * Z) + (xfm[1][3] * 1)
    Zt = (xfm[2][0] * X) + (xfm[2][1] * Y) + (xfm[2][2] * Z) + (xfm[2][3] * 1)
    1  = (xfm[3][0] * X) + (xfm[3][1] * Y) + (xfm[3][2] * Z) + (xfm[3][3] * 1)
```
You might be asking what the `1` in the 4th position of the point is for. The short answer is that it is the
translation multiplier: and is `1` for points, because they have a position; and `0` for vectors because they
are direction only, and not tied to a position. Thus, we can use the same transformation for vectors as:
```
     [ xfm[0][0] xfm[0][1] xfm[0][2] xfm[0][3] ] [ i ]   [ it ]
     | xfm[1][0] xfm[1][1] xfm[1][2] xfm[1][3] | | j | = | jt |
     | xfm[2][0] xfm[2][1] xfm[2][2] xfm[2][3] | | k |   | kt |
     [ xfm[3][0] xfm[3][1] xfm[3][2] xfm[3][3] ] [ 0 ]   [ 0  ]
```

or 
```
    it = (xfm[0][0] * i) + (xfm[0][1] * j) + (xfm[0][2] * k)
    jt = (xfm[1][0] * i) + (xfm[1][1] * j) + (xfm[1][2] * k) 
    kt = (xfm[2][0] * i) + (xfm[2][1] * j) + (xfm[2][2] * k)
    0  = 0
```

OK, that sounds cool, but, how does it relate to us and our robot. Well, the heading is a rotation
around the Z axis (NOTE, however, as described in the coordinate system part of the previous
section, we are using right-hand coordinate systems for the field and the robot - and in a right-hand
coordinate system a positive Z rotation rotates X into Y, or is counter-clockwise;
we have been using heading as a clockwise rotation, so we need to negate it for the vector geometry
formalization); and the position of the
robot on the field, call it `(Tx,Ty,Tz=0)` (for translation in X and Y) results in a transformation of:

```
     [  cos(-heading) sin(-heading) 0.0   Tx ]
     | -sin(-heading) cos(-heading) 0.0   Ty |
     |      0.0           0.0       1.0   Tz |
     [      0.0           0.0       0.0  1.0 ]
```

And when we walk through the transformation multipltation:
```
    Xt = (( cos(-heading) * X) + (sin(-heading) * Y) + 0.0 +  Tx)
    Yt = ((-sin(-heading) * X) + (cos(-heading) * Y) + 0.0 +  Ty)
    Zt = (       0.0           +       0.0           +  Z  +  Tz)
    1  = (       0.0           +       0.0           + 0.0 + 1.0)
```

and
```
    it = (( cos(-heading) * i) + (sin(-heading) * j) + 0.0)
    jt = ((-sin(-heading) * i) + (cos(-heading) * j) + 0.0) 
    kt = (       0.0           +        0.0      +      k)
    0  = 0
```

In the last section we wrote a bit of code to see what 3d points, lines, and vectors looked like in code. We
aren't going to write 3d transformation code - or use our previous, example code. Instead, I've added all of
that to our `a05annexUtil` library and we will be using that so the code we write here looks like the
code we would write for the robot. The next cell is the instructions to connect to that library. 

***FIRST*** *fun fact*: the robot is a new effort every season. Specifically, you are supposed to build a new robot and write new programming every season with one
specific exception: designs or code that are publically posted so they are available to other teams may be used
in the next seasons. This is why all of our code it on github (open source), and we have packaged and
released our `a05annexUtil` library so it can be automatically loaded from the open source maven repositories.

The `a05annexUtil` with the 3d vector geometry code is currently in a *staging* repository. This is a test
repository so we can use and test it here or on our robot before we officially release it to the Maven open
source repositories used by the *gradle* build tools. The `<repository>`
section of the next cell describes where the repository is on the internet, and the `<dependency>` section
describes our `a05annexUtil` in that repository:

In [3]:
%%loadFromPOM
<repository>
  <id>oss-sonatype-staging</id>
  <url>https://oss.sonatype.org/content/repositories/Releases/</url>
</repository>

<dependency>
  <groupId>org.a05annex</groupId>
  <artifactId>a05annexUtil</artifactId>
  <version>0.9.1</version>
</dependency>

When you run the cell above you should not get any output, if you do get output, that means there is a problem. For
example, if you change the version to `0.8.10` and run the cell there will be an error `[unresolved dependency: org.a05annex#a05annexUtil;0.8.10: not found]` because there is no version `0.8.10`. **NOTE:** once we are done testing and release this library it wil be in the *releases* area rather than *staging*.

OK, let's do a little test to make sure we can get to the `a05annexUtil` 3d vector math classes:

In [4]:
import org.a05annex.util.geo3d.Point3d;
import org.a05annex.util.geo3d.Vector3d;

// create a couple points
Point3d pt0 = new Point3d(0.0, 0.0, 0.0);
Point3d pt1 = new Point3d(1.0, 2.0, 3.0);
// get the direction vector from pt0 to pt1
Vector3d v = new Vector3d(pt0,pt1).normalize();
// and print out some info to verify
System.out.println("pt0: (" + pt0.getX() + "," + pt0.getY() + "," + pt0.getZ() + ")");
System.out.println("pt1: (" + pt1.getX() + "," + pt1.getY() + "," + pt1.getZ() + ")");
System.out.println("v: (" + v.getI() + "," + v.getJ() + "," + v.getK() + ")");
System.out.println("Length v is: " + v.getLength());


pt0: (0.0,0.0,0.0)
pt1: (1.0,2.0,3.0)
v: (0.2672612341463735,0.534522468292747,0.8017837024391204)
Length v is: 1.0


Excellent, we are using the new library and we are ready to do some really cool stuff.

### Field to/from Robot Transformations

Whoa - that last section was a lot to digest, so let's look a transformation as it relates to us.
This section will describe transformation from robot coordinates to field coordinates, and from
field coordinates to robot coordinates. In our 2020-2021 swerve drive code we added code to
track where we think the robot is on the field
based on our control commands - this does not account for slippage on the field or collisions with
other robots. In our path planning we transformed the robot coordinates (the geometry of the robot)
to field coordinates to display robot position on the field and to test for the robot extending past
field boundaries. In our exploration of trying to use field elements to determine where the robot
is on the field we determine where the field elements are relative to the robot and work backwards
from that to figure out where where the robot is on the field.

Let's look at the
infinite recharge field; specifically the high port in the scoring goal with the large hexagonal primary
front face goal that has the limelight tape, and the smaller circular goal that is centered on the
hexagon; 1'-1" in diameter and 2'-5&frac14; (0.74m)" behind the primary front face. So, let us diagram that
remembering that our convention for path planning is that the origin of the field coordinate
system, `(0.0, 0.0, 0.0)`, is the field center. We have a several standard start positions (though these
are normally the back bumper of the robot) midfield at
the start line `(0.0, 5.0, 0.0)`, right (in front of goal) at the start line `(1.7085, 5.0, 0.0)`, and left
(in front of the ball pickup) at `(-1.56, 5.0, 0.0)`. For shooting from the end of the trench to protect us from defense, the location is `(2.695, 2.743, 0.0)`:

![alt text](./resources/InfiniteRechargeField.jpg "infinite recharge field")


Where do we go from here? Lets start by building a cell that defines the key points of the playing field:

In [77]:
import org.a05annex.util.geo3d.Point3d;
import org.a05annex.util.geo3d.Vector3d;

static final int FIELD_CENTER = 0;
static final int START_CENTER = 1;
static final int START_RIGHT = 2;
static final int START_LEFT = 3;
static final int TRENCH_CORNER = 4;
static final int HEX_GOAL_BASE = 5;
static final int HEX_GOAL_CENTER = 6;
static final int HEX_GOAL_LL_TAPE_UL = 7;
static final int HEX_GOAL_LL_TAPE_LR = 8;
static final int HEX_GOAL_TAPE_CENTROID = 9;
static final int CENTER_GOAL = 10;
static final int HEX_OP_GOAL_BASE = 11;
static final int HEX_OP_GOAL_CENTER = 12;
static final int HEX_OP_GOAL_LL_TAPE_UL = 13;
static final int HEX_OP_GOAL_LL_TAPE_LR = 14;
static final int HEX_OP_GOAL_TAPE_CENTROID = 15;
static final String[] FIELD_NAMES = {"field center:         ",
                                     "start center:         ",
                                     "start right:          ",
                                     "start left:           ",
                                     "trench corner:        ",
                                     "hex goal base:        ",
                                     "hex goal center:      ",
                                     "hex goal tape UL:     ",
                                     "hex goal tape LR:     ",
                                     "hex goal tape center: ",
                                     "round goal center:    ",
                                     "opp hex goal base:    ",
                                     "opp hex goal center:  ",
                                     "opp goal tape UL:     ",
                                     "opp goal tape LR::    ",
                                     "opp goal tape center: "};
static final Point3d[] FIELD_POINTS = {new Point3d( 0.0, 0.0, 0.0),
                                       new Point3d( 0.0, 5.0, 0.0),
                                       new Point3d( 1.7085, 5.0, 0.0),
                                       new Point3d(-1.56, 5.0, 0.0),
                                       new Point3d( 2.695, 2.743, 0.0),
                                       new Point3d( 1.7085, 7.99, 0.0),
                                       new Point3d( 1.7085, 7.99, 2.5),
                                       new Point3d( 1.21, 7.99, 2.5),
                                       new Point3d( 2.1870, 7.99, 2.0625),
                                       new Point3d( 1.7085, 7.99, 2.2841),
                                       new Point3d( 1.7085, 8.73, 2.5),
                                       new Point3d(-1.7085, -7.99, 0.0),
                                       new Point3d(-1.7085, -7.99, 2.5),
                                       new Point3d(-1.21, -7.99, 2.5),
                                       new Point3d(-2.1870, -7.99, 2.0625),
                                       new Point3d(-1.7085, -7.99, 2.2851)};

/**
 * A utility method to print a point
 */
void printPoint(String title, Point3d pt) {
    System.out.println(String.format("%s (%7.3f,%7.3f,%7.3f)", title, pt.getX(), pt.getY(), pt.getZ()));
}

/**
 * A utility method to print a vector
 */
void printVector(String title, Vector3d v) {
    System.out.println(String.format("%s (%7.3f,%7.3f,%7.3f)", title, v.getI(), v.getJ(), v.getK()));
}

/**
 * A utility method for printing an array of points.
 */
public void printPoints(String[] titles, Point3d[] pts)
{
    for (int i = 0; i < pts.length; i++) {
        printPoint(titles[i], pts[i]);
    }
}

/**
 * A utility method for printing an array of vectors.
 */
public void printVectors(String[] titles, Vector3d[] vecs)
{
    for (int i = 0; i < vecs.length; i++) {
        printVector(titles[i], vecs[i]);
    }
}

System.out.println("-- vector print test --");
Vector3d testVector = new Vector3d(1.0,2.0,3.0);
printVector("test vector:           ",testVector);
printVector("normalized test vector:",testVector.normalize());
System.out.println("-- field locations --");
printPoints(FIELD_NAMES, FIELD_POINTS);

-- vector print test --
test vector:            (  1.000,  2.000,  3.000)
normalized test vector: (  0.267,  0.535,  0.802)
-- field locations --
field center:          (  0.000,  0.000,  0.000)
start center:          (  0.000,  5.000,  0.000)
start right:           (  1.709,  5.000,  0.000)
start left:            ( -1.560,  5.000,  0.000)
trench corner:         (  2.695,  2.743,  0.000)
hex goal base:         (  1.709,  7.990,  0.000)
hex goal center:       (  1.709,  7.990,  2.500)
hex goal tape UL:      (  1.210,  7.990,  2.500)
hex goal tape LR:      (  2.187,  7.990,  2.063)
hex goal tape center:  (  1.709,  7.990,  2.284)
round goal center:     (  1.709,  8.730,  2.500)
opp hex goal base:     ( -1.709, -7.990,  0.000)
opp hex goal center:   ( -1.709, -7.990,  2.500)
opp goal tape UL:      ( -1.210, -7.990,  2.500)
opp goal tape LR::     ( -2.187, -7.990,  2.063)
opp goal tape center:  ( -1.709, -7.990,  2.285)


Next we create a class that represents the limelight, specifically the geometry of the limelight relative
to some other axis system or frame or reference - typically the robot. Our `Limelight` class will save
the position of the camera on the robot; the transformation from robot to limelight and from limelight to robot;
and some test geometry we can use to test whether the transformations are working. Additionally, to make
this a versatile as possible since we don't know how the limelight is mounted (could be mounted to an arm that moves), we will imagine:
* it is mounted on a turntable, so it can be turned to face anywhere on the field
* its mount on the turntable can tilt the camera up or down
* its position can be changed throughout the match

This means we need a set limelight position method we can call anytime to reset the position of the limelight on the robot.

The main challenge here is figuring out how to build the transformation from the robot coordinate
system to the limelight coordinate system. To do this we start by imaginging the limelight with the XY axes
of the limelight matching the XY axis of the robot (using a physical robot and camera is a good way to do
this). In this imagining:
* the camera starts lens-down on the center of the robot with the top of the limelight
  facing the front of the robot - limelight Y is towards the top of the screen and we are matching it to robot
  Y which is towards the front of the robot. the X of the camera is towards the right side of the camera
  aligned with the X axis pf the tobot.
* The limelight is a left-handed coorinate system, so +Z is in front of the lens, or down. The robot and
  field are right handed coordinate system, so +Z is up. So we scale Z by -1 to reverse its direction
  to match it's direction to the robot.
* Next, we would like the reference position for the limelight to be facing the front of the robot, so we need
  a `&pi;/2.0` (`90%deg;`) rotation around X (a positive rotation rotates the Y axis into the Z axis).
* At this point X axis of the camera matches the X axis of the robot so we can apply the elevation angle
  as a rotation on X (we can add it to the X rotation of the last step.
* At this point the +Z axis of the robot is the axis of rotation of
  the turntable. If we think about expressing turnbale position relative to the robot similarly to robot
  heading then we apply the `-limelightHeading` as a rotation on Z.
  
  OK, lets write some code for this:

In [30]:
import org.a05annex.util.AngleD;
import org.a05annex.util.geo3d.Point3d;
import org.a05annex.util.geo3d.Vector3d;
import org.a05annex.util.geo3d.Xfm4x4d;

class Limelight {
    
    // The limelight coordinates
    static final int CENTER = 0;
    static final int UNIT_X = 1;
    static final int UNIT_Y = 2;
    static final int UNIT_Z = 3;

    static final String[] LIMELIGHT_PT_NAMES = {"center:"};
    static final String[] LIMELIGHT_V_NAMES = {"unit X:",
                                               "unit Y:",
                                               "unit Z:"};
    static final Point3d[] LIMELIGHT_PT_GEOMETRY = {new Point3d(0.0, 0.0, 0.0)};
    static final Vector3d[] LIMELIGHT_V_GEOMETRY = {new Vector3d(1.0, 0.0, 0.0),
        new Vector3d(0.0, 1.0, 0.0), new Vector3d(0.0, 0.0, 1.0)};
    
    double m_robotX = 0.0;
    double m_robotY = 0.0;
    double m_robotZ = 0.0;
    final AngleD m_heading = new AngleD(AngleD.RADIANS, 0.0);
    final AngleD m_elevation = new AngleD(AngleD.RADIANS, 0.0);
    
    Xfm4x4d m_limelightToRobotXfm = new Xfm4x4d();
    Xfm4x4d m_robotToLimelightXfm = new Xfm4x4d();
    
    public Limelight()
    {
        setLimelightPosition(0.0, 0.0, 0.0,
                             new AngleD(AngleD.DEGREES,0.0), new AngleD(AngleD.DEGREES,0.0));
    }
    
    public Limelight(double robotX, double robotY, double robotZ, AngleD heading, AngleD elevation)
    {
        setLimelightPosition(robotX, robotY, robotZ, heading, elevation);
    }
    
    /**
     * Set the position of the limelight on the robot.
     */
    public void setLimelightPosition(double robotX, double robotY, double robotZ,
        AngleD heading, AngleD elevation)
    {
        m_robotX = robotX;
        m_robotY = robotY;
        m_robotZ = robotZ;
        m_heading.setValue(heading);
        m_elevation.setValue(elevation);
        // build the transform that positions the limelight relative to the robot
        // by initializing the transform to an identify, applying a scaling to map
        // from a left handed coordinate system to a right handed, a rotation to
        // set the elevation angle, and a translation to set position of the limelight
        // on the robot.
        m_limelightToRobotXfm.identity();
        // rescale the axis system from a left handed to right handed axis system
        m_limelightToRobotXfm.scale(1.0, 1.0, -1.0);
        // rotate around X until the Y is straight up (90deg) plus the tilt angle for the
        // camera facing forward
        AngleD xRotation = new AngleD(AngleD.DEGREES, 90.0).add(m_elevation);
        m_limelightToRobotXfm.rotate(Xfm4x4d.AXIS_X, xRotation);
        // The turntable rotation in robot Z
        m_limelightToRobotXfm.rotate(Xfm4x4d.AXIS_Z, new AngleD(heading).mult(-1.0));
        // And finally the translation that places the limelight on the robot.
        m_limelightToRobotXfm.translate(m_robotX, m_robotY, m_robotZ);
        // invert that to buld the transform that describes where the robot
        // elements are relative to the limelight
        m_limelightToRobotXfm.invert(m_robotToLimelightXfm);
    }
    
    /**
     * Transform a robot-relative point to limelight-relative
     * point. This takes a robot-relative point, and
     * creates a limelight-relative point. The robot-relative
     * point is unchanged.
     */
    public Point3d xfmRobotPtToLimelight(Point3d robotPt)
    {
        return m_robotToLimelightXfm.transform(robotPt, new Point3d());
    }
    
    /**
     * Transform an array of robot-relative points to limelight-relative
     * points. This takes an array of robot-relative points, and
     * creates an array of limelight-relative points. The robot-relative
     * points are unchanged.
     */
    public Point3d[] xfmRobotPtsToLimelight(Point3d[] robotPts)
    {
        return m_robotToLimelightXfm.cloneAndTransform(robotPts);
    }
    
    /**
     * Transform a robot-relative vector to limelight-relative
     * vector. This takes a robot-relative vector, and
     * creates a limelight-relative vector. The robot-relative
     * vector is unchanged.
     */
    public Vector3d xfmRobotVecToLimelight(Vector3d robotV)
    {
        return m_robotToLimelightXfm.transform(robotV, new Vector3d());
    }
    
    /**
     * Transform an array of robot-relative vectors to limelight-relative
     * vectors. This takes an array of robot-relative vectors, and
     * creates an array of limelight-relative vectors. The robot-relative
     * vectors are unchanged.
     */
    public Vector3d[] xfmRobotVecsToLimelight(Vector3d[] robotVs)
    {
        return m_robotToLimelightXfm.cloneAndTransform(robotVs);
    }
    
    /**
     * Transform a limelight-relative point to robot-relative point and
     * return that robot-relative point. This is useful if you want to
     * display the limelight on a diagram of the robot.
     */
    public Point3d xfmLimelightPtToRobot(Point3d limelightPt)
    {
        return m_limelightToRobotXfm.transform(limelightPt, new Point3d());
    }
    
    /**
     * Transform the limelight-relative point geometry to robot-relative points
     * and return those robot-relative points. This is useful if you want to
     * display the limelight on a diagram of the robot.
     */
    public Point3d[] xfmLimelightPtGeoToRobot()
    {
        return m_limelightToRobotXfm.cloneAndTransform(LIMELIGHT_PT_GEOMETRY);
    }
    
    /**
     * Transform a limelight-relative vector to robot-relative vector and
     * return the robot-relative vector. This is useful if you want to
     * display the limelight on a diagram of the robot.
     */
    public Vector3d xfmLimelightVecToRobot(Vector3d limelightV)
    {
        return m_limelightToRobotXfm.transform(limelightV, new Vector3d());
    }
    
    /**
     * Transform the limelight-relative vector geometry to robot-relative vectors
     * and returns those robot-relative vectors. This is useful if you want to
     * display the limelight on a diagram of the robot.
     */
    public Vector3d[] xfmLimelightVecGeoToRobot()
    {
        return m_limelightToRobotXfm.cloneAndTransform(LIMELIGHT_V_GEOMETRY);
    }
    
    /**
     * Print an annotated list of the limelight geometry transformed to robot-relative
     * coordinates for the current camera position. This is primarily a debugging method.
     */
    public void printLimelightGeomToRobot()
    {
        printPoints(LIMELIGHT_PT_NAMES, xfmLimelightPtGeoToRobot());
        printVectors(LIMELIGHT_V_NAMES, xfmLimelightVecGeoToRobot());
    }
}


OK, let's check this out a little bit to see it it works. We will start by puting the camera at (0,0,0) with
no rotation or elevation then we will get the camera geometry in robot coordinates, and transform it back to
make sure it transforms correctly back to the orginal camera position. Then repeat that for a: 90&deg;
rotation; a 90&deg; elevation; and both a 90&deg; rotation and elevation:

In [31]:
Limelight limelight = new Limelight();


System.out.println("-- limelight at robot (0.0,0.0,0.0) rotate=0.0, elevation = 0.0 --");
System.out.println("------------------------------------------------------------------");
System.out.println("-- camera geometry on robot ==");
limelight.printLimelightGeomToRobot();
System.out.println("-- xfm from robot back to camera ==");
printPoints(Limelight.LIMELIGHT_PT_NAMES,
    limelight.xfmRobotPtsToLimelight(limelight.xfmLimelightPtGeoToRobot()));
printVectors(Limelight.LIMELIGHT_V_NAMES,
    limelight.xfmRobotVecsToLimelight(limelight.xfmLimelightVecGeoToRobot()));



-- limelight at robot (0.0,0.0,0.0) rotate=0.0, elevation = 0.0 --
------------------------------------------------------------------
-- camera geometry on robot ==
center: (  0.000,  0.000,  0.000)
unit X: (  1.000,  0.000,  0.000)
unit Y: (  0.000,  0.000,  1.000)
unit Z: (  0.000,  1.000, -0.000)
-- xfm from robot back to camera ==
center: (  0.000,  0.000,  0.000)
unit X: (  1.000,  0.000,  0.000)
unit Y: (  0.000,  1.000,  0.000)
unit Z: (  0.000,  0.000,  1.000)


And we will put that into a little method that we might use later when testing and quick verify the other 3 cases:

In [32]:
Limelight limelight = new Limelight();

void limelightGeoFor(double robotX, double robotY, double robotZ,
        AngleD heading, AngleD elevation) {
    limelight.setLimelightPosition(robotX, robotY, robotZ, heading, elevation);
    System.out.println(String.format(
        "\n-- limelight at robot (%.2f0,%.2f,%.2f) rotate=%.2f0, elevation=%.2f --",
        robotX, robotY, robotZ, heading.getDegrees(), elevation.getDegrees()));
    System.out.println("-------------------------------------------------------------------------------");
    System.out.println("-- camera geometry on robot ==");
    limelight.printLimelightGeomToRobot();
    System.out.println("-- xfm from robot back to camera ==");
    printPoints(Limelight.LIMELIGHT_PT_NAMES,
        limelight.xfmRobotPtsToLimelight(limelight.xfmLimelightPtGeoToRobot()));
    printVectors(Limelight.LIMELIGHT_V_NAMES,
        limelight.xfmRobotVecsToLimelight(limelight.xfmLimelightVecGeoToRobot()));
}

limelightGeoFor(0.0, 0.0, 0.0, new AngleD(AngleD.DEGREES,0.0), new AngleD(AngleD.DEGREES,0.0));
limelightGeoFor(0.0, 0.0, 0.0, new AngleD(AngleD.DEGREES,90.0), new AngleD(AngleD.DEGREES,0.0));
limelightGeoFor(0.0, 0.0, 0.0, new AngleD(AngleD.DEGREES,0.0), new AngleD(AngleD.DEGREES,90.0));
limelightGeoFor(0.0, 0.0, 0.0, new AngleD(AngleD.DEGREES,90.0), new AngleD(AngleD.DEGREES,90.0));
limelightGeoFor(0.2, -0.3, 0.5, new AngleD(AngleD.DEGREES,0.0), new AngleD(AngleD.DEGREES,0.0));


-- limelight at robot (0.000,0.00,0.00) rotate=0.000, elevation=0.00 --
-------------------------------------------------------------------------------
-- camera geometry on robot ==
center: (  0.000,  0.000,  0.000)
unit X: (  1.000,  0.000,  0.000)
unit Y: (  0.000,  0.000,  1.000)
unit Z: (  0.000,  1.000, -0.000)
-- xfm from robot back to camera ==
center: (  0.000,  0.000,  0.000)
unit X: (  1.000,  0.000,  0.000)
unit Y: (  0.000,  1.000,  0.000)
unit Z: (  0.000,  0.000,  1.000)

-- limelight at robot (0.000,0.00,0.00) rotate=90.000, elevation=0.00 --
-------------------------------------------------------------------------------
-- camera geometry on robot ==
center: (  0.000,  0.000,  0.000)
unit X: (  0.000, -1.000,  0.000)
unit Y: (  0.000,  0.000,  1.000)
unit Z: (  1.000,  0.000, -0.000)
-- xfm from robot back to camera ==
center: (  0.000,  0.000,  0.000)
unit X: (  1.000,  0.000,  0.000)
unit Y: (  0.000,  1.000,  0.000)
unit Z: (  0.000,  0.000,  1.000)

-- limelight a

And this works as expected, specifically the camera location remains at (0,0,0) unless we specifically
move it, so we can use this new class to test what happens when the camera has an elevation angle like
it does on our current robot.

## In session 12, we ended with a question about camera tilt affects limelight angles.

The question was if the camera is tilted, how does that affect the limelight Y angle when the
target is off-center. And we didn't have a good way to compute an answer for that:
![alt text](./resources/ElevatedLimelight.jpg "simple distance with limelight")

Let's go back to the simple robot camera geometry drawing:
![alt text](./resources/limelight_side_geom.jpg "limelight side view")

We can use our newly acquired vector math skills to answer that question. Start by imagining the camera centered on the robot with the correct height and elevation. The height is .52m, and the angle is .337rad. The limelight target centroid height is
2.2851m so if we put the target `(2.2851m - .52m)/tan(.337) = 5.0379m` in front of the robot, the limelight should be centered on the target.

We can do this pretty simply by pretending the field is the robot and placing the camera 5.6517m back from the target, at the height and elevation it would be on the robot:

In [33]:
limelight.setLimelightPosition(
    FIELD_POINTS[HEX_GOAL_TAPE_CENTROID].getX(),
    FIELD_POINTS[HEX_GOAL_TAPE_CENTROID].getY() - 5.0379, 0.520,
    new AngleD(AngleD.RADIANS,0.0), new AngleD(AngleD.RADIANS,0.337));
    
Point3d targetLocation = limelight.xfmRobotPtToLimelight(FIELD_POINTS[HEX_GOAL_TAPE_CENTROID]);
printPoint("target location in limelight coordinates: ", targetLocation);

target location in limelight coordinates:  (  0.000, -0.001,  5.338)


So now, we can just rotate the camera on the turntable to see what would happen if the robot rotates:

In [34]:
for (double degrees = -90.0; degrees <= 95.0; degrees += 10.0) {
    limelight.setLimelightPosition(
        FIELD_POINTS[HEX_GOAL_TAPE_CENTROID].getX(),
        FIELD_POINTS[HEX_GOAL_TAPE_CENTROID].getY() -5.6517, 0.520,
        new AngleD(AngleD.DEGREES,degrees), new AngleD(AngleD.RADIANS,0.337));
    Point3d targetLocation = limelight.xfmRobotPtToLimelight(
        FIELD_POINTS[HEX_GOAL_TAPE_CENTROID]);
    System.out.println(
        String.format("rotation: %7.3fdeg; limelight XYZ: (%7.3f,%7.3f,%7.3f)",
        degrees, targetLocation.getX(),targetLocation.getY(),targetLocation.getZ()));
}

rotation: -90.000deg; limelight XYZ: (  5.652,  1.665,  0.583)
rotation: -80.000deg; limelight XYZ: (  5.566,  1.340,  1.510)
rotation: -70.000deg; limelight XYZ: (  5.311,  1.026,  2.408)
rotation: -60.000deg; limelight XYZ: (  4.895,  0.730,  3.250)
rotation: -50.000deg; limelight XYZ: (  4.329,  0.464,  4.012)
rotation: -40.000deg; limelight XYZ: (  3.633,  0.233,  4.669)
rotation: -30.000deg; limelight XYZ: (  2.826,  0.046,  5.203)
rotation: -20.000deg; limelight XYZ: (  1.933, -0.091,  5.595)
rotation: -10.000deg; limelight XYZ: (  0.981, -0.176,  5.836)
rotation:   0.000deg; limelight XYZ: (  0.000, -0.204,  5.917)
rotation:  10.000deg; limelight XYZ: ( -0.981, -0.176,  5.836)
rotation:  20.000deg; limelight XYZ: ( -1.933, -0.091,  5.595)
rotation:  30.000deg; limelight XYZ: ( -2.826,  0.046,  5.203)
rotation:  40.000deg; limelight XYZ: ( -3.633,  0.233,  4.669)
rotation:  50.000deg; limelight XYZ: ( -4.329,  0.464,  4.012)
rotation:  60.000deg; limelight XYZ: ( -4.895,  0.730, 

Well, this is interesting, but not very useful, what we really need to know is what X and Y angles the
limelight will report. Fortunately, the coordinates of the points project into triangles on the X amd Y
planes (because the origin of the camera is (0,0,0)) and we can easily turn the target location into angles
using the `atan2()` function:


In [35]:
// start with the location when camera is rotated 30 degrees
Point3d targetLocation = new Point3d(-2.826,  0.046,  5.203);
// use atan2 to get the angles
AngleD limelightX = new AngleD().atan2(targetLocation.getX(),targetLocation.getZ());
AngleD limelightY = new AngleD().atan2(targetLocation.getY(),targetLocation.getZ());
System.out.println(String.format("limelight X: %7.3f limelight Y: %7.3f",
    limelightX.getDegrees(), limelightY.getDegrees()));

limelight X: -28.509 limelight Y:   0.507


OK, now we know how to get where the target is seen on the screen for any camera position on the
field, but how do we work the other way? Knowing the limelght XY angles, how do we get the
position on the field? Well we use a little trig to imagine going 1.0 from the lens along the
limelight Z and finding out the point where the line in the XY angle direction hits the plane.
This gives us a start and end point so we can build a direction vector from the camera to the target.
If we transform that back to robot space we can get the get the heading and elevation angles between the
robot and the target.

In [36]:
for (double degrees = -90.0; degrees <= 95.0; degrees += 10.0) {
    limelight.setLimelightPosition(
        FIELD_POINTS[HEX_GOAL_TAPE_CENTROID].getX(),
        FIELD_POINTS[HEX_GOAL_TAPE_CENTROID].getY() -5.6517, 0.520,
        new AngleD(AngleD.DEGREES,degrees), new AngleD(AngleD.RADIANS,0.337));
    Point3d targetLocation = limelight.xfmRobotPtToLimelight(
        FIELD_POINTS[HEX_GOAL_TAPE_CENTROID]);
    // use atan2 to get the angles
    AngleD limelightX = new AngleD().atan2(targetLocation.getX(),targetLocation.getZ());
    AngleD limelightY = new AngleD().atan2(targetLocation.getY(),targetLocation.getZ());
    Vector3d limelightToTarget = new Vector3d(limelightX.tan(), limelightY.tan(), 1.0).normalize();
    Vector3d robotToTarget = limelight.xfmLimelightVecToRobot(limelightToTarget);
    AngleD heading = new AngleD().atan2(robotToTarget.getI(),robotToTarget.getJ());
    AngleD elevation = new AngleD().asin(robotToTarget.getK());
    System.out.println(String.format(
        "rotation: %7.3fdeg; limelight XY: %7.3fdeg, %7.3fdeg;  heading: %7.3fdeg, elev: %7.3frad",
        degrees, limelightX.getDegrees(), limelightY.getDegrees(),
        heading.getDegrees(), elevation.getRadians()));
}

rotation: -90.000deg; limelight XY:  84.107deg,  70.691deg;  heading:   0.000deg, elev:   0.303rad
rotation: -80.000deg; limelight XY:  74.826deg,  41.603deg;  heading:   0.000deg, elev:   0.303rad
rotation: -70.000deg; limelight XY:  65.614deg,  23.076deg;  heading:   0.000deg, elev:   0.303rad
rotation: -60.000deg; limelight XY:  56.414deg,  12.667deg;  heading:   0.000deg, elev:   0.303rad
rotation: -50.000deg; limelight XY:  47.181deg,   6.592deg;  heading:   0.000deg, elev:   0.303rad
rotation: -40.000deg; limelight XY:  37.884deg,   2.860deg;  heading:  -0.000deg, elev:   0.303rad
rotation: -30.000deg; limelight XY:  28.509deg,   0.512deg;  heading:   0.000deg, elev:   0.303rad
rotation: -20.000deg; limelight XY:  19.058deg,  -0.934deg;  heading:   0.000deg, elev:   0.303rad
rotation: -10.000deg; limelight XY:   9.546deg,  -1.723deg;  heading:  -0.000deg, elev:   0.303rad
rotation:   0.000deg; limelight XY:   0.000deg,  -1.974deg;  heading:   0.000deg, elev:   0.303rad
rotation: 

This is an interesting result, since the camera is positioned as though it rotating on a stationary robot parked in front of the target the computed heading to the target and height should not channge as the camera is rotated.

As a last step we need to compute the location of the robot (the camera) on the field. to do that we use the elevation angle to compute the distance as we did in session 11, use the heading to compute a direction vector,
and then fome from the target that distance in the right direction.

To demonstrate this, lets create a collection of points 1m away from the the test point we've been using and then use them with the camera always facing forward, and then modify our loop to go through those points:

In [37]:
import org.a05annex.util.geo3d.Line3d;

double xCenter = FIELD_POINTS[HEX_GOAL_TAPE_CENTROID].getX();
double yCenter = FIELD_POINTS[HEX_GOAL_TAPE_CENTROID].getY() - 5.0379;
double zCenter = 0.520;
Point3d testPts[] = {new Point3d(xCenter - 1.0, yCenter - 1.0, 0.520),
                     new Point3d(xCenter      , yCenter - 1.0, 0.520),
                     new Point3d(xCenter + 1.0, yCenter - 1.0, 0.520),
                     new Point3d(xCenter - 1.0, yCenter      , 0.520),
                     new Point3d(xCenter      , yCenter      , 0.520),
                     new Point3d(xCenter + 1.0, yCenter      , 0.520),
                     new Point3d(xCenter - 1.0, yCenter + 1.0, 0.520),
                     new Point3d(xCenter      , yCenter + 1.0, 0.520),
                     new Point3d(xCenter + 1.0, yCenter + 1.0, 0.520)
};

for(int i = 0; i < 9; i++) {
    limelight.setLimelightPosition(testPts[i].getX(),testPts[i].getY(), testPts[i].getZ(),
        new AngleD(AngleD.DEGREES,0.0), new AngleD(AngleD.RADIANS,0.337));
    Point3d targetLocation = limelight.xfmRobotPtToLimelight(FIELD_POINTS[HEX_GOAL_TAPE_CENTROID]);
    // printPoint("target location in limelight coordinates: ", targetLocation);
    // use atan2 to get the angles
    AngleD limelightX = new AngleD().atan2(targetLocation.getX(),targetLocation.getZ());
    AngleD limelightY = new AngleD().atan2(targetLocation.getY(),targetLocation.getZ());
    Vector3d limelightToTarget = new Vector3d(limelightX.tan(), limelightY.tan(), 1.0).normalize();
    Vector3d robotToTarget = limelight.xfmLimelightVecToRobot(limelightToTarget);
    AngleD heading = new AngleD().atan2(robotToTarget.getI(),robotToTarget.getJ());
    AngleD elevation = new AngleD().asin(robotToTarget.getK());
    double targetDistance = (FIELD_POINTS[HEX_GOAL_TAPE_CENTROID].getZ() - testPts[i].getZ()) / elevation.sin();
    Vector3d targetToLimelight = robotToTarget.cloneVector3d().reverse();
    Point3d fieldPt = new Line3d(FIELD_POINTS[HEX_GOAL_TAPE_CENTROID],
                                 targetToLimelight).pointAtDistance(targetDistance);               
    System.out.println(String.format(
        "(%7.3f,%7.3f,%7.3f); limelight XY: %7.3fdeg, %7.3fdeg; (%7.3f,%7.3f,%7.3f)",
        testPts[i].getX(),testPts[i].getY(), testPts[i].getZ(),
        limelightX.getDegrees(), limelightY.getDegrees(),
        fieldPt.getX(), fieldPt.getY(), fieldPt.getZ()));

}

(  0.708,  1.952,  0.520); limelight XY:   9.045deg,  -3.022deg; (  0.709,  1.952,  0.520)
(  1.709,  1.952,  0.520); limelight XY:   0.000deg,  -3.022deg; (  1.709,  1.952,  0.520)
(  2.709,  1.952,  0.520); limelight XY:  -9.045deg,  -3.022deg; (  2.709,  1.952,  0.520)
(  0.708,  2.952,  0.520); limelight XY:  10.611deg,  -0.010deg; (  0.709,  2.952,  0.520)
(  1.709,  2.952,  0.520); limelight XY:   0.000deg,  -0.010deg; (  1.709,  2.952,  0.520)
(  2.709,  2.952,  0.520); limelight XY: -10.611deg,  -0.010deg; (  2.708,  2.952,  0.520)
(  0.708,  3.952,  0.520); limelight XY:  12.821deg,   4.291deg; (  0.709,  3.952,  0.520)
(  1.709,  3.952,  0.520); limelight XY:   0.000deg,   4.291deg; (  1.709,  3.952,  0.520)
(  2.709,  3.952,  0.520); limelight XY: -12.821deg,   4.291deg; (  2.708,  3.952,  0.520)


The obvious next steps are:
* Take that last bit of code and work it into the camera class so there are methods to:
  * predict the target angles on the limelight - for testing and calibration.
  * convert from limelight target angles to a targetToLimelight direction vector and
    target to limelight distance
  * support multiple for multiple targets (my goal, opposing goal)
* Create a Robot class so we have something to mount the camera on

In [70]:
import org.a05annex.util.geo3d.Line3d;
import org.a05annex.util.geo3d.Point3d;
import org.a05annex.util.geo3d.Vector3d;
import org.a05annex.util.geo3d.Xfm4x4d;
import org.a05annex.util.AngleD;

/**
 * A class preresenting the Limelight geometry and positioning of the limelight on
 * the robot.
 */
class Limelight {
    
    /**
     * A class representing a limelight target on the field
     */
    private class Target {
        final String m_identifier;
        final Point3d m_centroid;
        final Vector3d m_normal;
        final double m_width;
        final double m_height;
        
        Target(String identifier, Point3d centroid, Vector3d normal, double width, double height)
        {
            m_identifier = identifier;
            m_centroid = centroid;
            m_normal = normal;
            m_width = width;
            m_height = height;
        }
        
        public void printTarget()
        {
            System.out.println(String.format("------ target '%s' ------", m_identifier));
            System.out.println(String.format("  centroid: (%7.3f,%7.3f,%7.3f)",
                                             m_centroid.getX(), m_centroid.getY(), m_centroid.getZ()));
            System.out.println(String.format("  normal:   (%7.3f,%7.3f,%7.3f)",
                                             m_normal.getI(), m_normal.getJ(), m_normal.getK()));
            System.out.println(String.format("  width:    %7.3f", m_width));
            System.out.println(String.format("  height:   %7.3f", m_height));
        }
    }
    
    /**
     * This is a class that represents the position of the target with respect to the robot. It is generated by
     * the {@link LimelightTelemetry} class and is a data structure only.
     */
    public class TargetPosition {
        final public String targetId;
        final public Vector3d robotToTarget;
        final public AngleD targetHeading;
        final public AngleD targetElevation;
        final public double targetDistance;
        final public Point3d limelightCenter;

        /**
         *
         * @param robotToTarget
         * @param targetHeading
         * @param targetElevation
         * @param targetDistance
         * @param limelightCenter
         */
        TargetPosition(String targetId, Vector3d robotToTarget,
                       AngleD targetHeading, AngleD targetElevation,
                       double targetDistance, Point3d limelightCenter) {
            this.targetId = targetId;
            this.robotToTarget = robotToTarget;
            this.targetHeading = targetHeading;
            this.targetElevation = targetElevation;
            this.targetDistance = targetDistance;
            this.limelightCenter = limelightCenter;
        }
        public void printTargetPosition()
        {
            System.out.println(String.format("------ target position for '%s' ------", targetId));
            System.out.println(String.format(
                "  robot to target: (%7.3f,%7.3f,%7.3f)",
                robotToTarget.getI(), robotToTarget.getJ(), robotToTarget.getK()));
            System.out.println(String.format(
                "  target heading:    %7.3f", targetHeading.getDegrees()));
            System.out.println(String.format(
                "  target elevation:  %7.3f", targetElevation.getDegrees()));
            System.out.println(String.format(
                "  target_distance:   %7.3f", targetDistance));
        }
    }
    
    // The limelight coordinates
    static private final int CENTER = 0;
    static private final int UNIT_X = 1;
    static private final int UNIT_Y = 2;
    static private final int UNIT_Z = 3;

    static private final String[] LIMELIGHT_PT_NAMES = {"center:"};
    static private final String[] LIMELIGHT_V_NAMES = {"unit X:",
                                                       "unit Y:",
                                                       "unit Z:"};
    static private final Point3d[] LIMELIGHT_PT_GEOMETRY = {new Point3d(0.0, 0.0, 0.0)};
    static private final Vector3d[] LIMELIGHT_V_GEOMETRY = {new Vector3d(1.0, 0.0, 0.0),
        new Vector3d(0.0, 1.0, 0.0), new Vector3d(0.0, 0.0, 1.0)};
    
    private double m_robotX = 0.0;
    private double m_robotY = 0.0;
    private double m_robotZ = 0.0;
    private final AngleD m_heading = new AngleD(AngleD.RADIANS, 0.0);
    private final AngleD m_elevation = new AngleD(AngleD.RADIANS, 0.0);
    
    private Xfm4x4d m_limelightToRobotXfm = new Xfm4x4d();
    private Xfm4x4d m_robotToLimelightXfm = new Xfm4x4d();
    
    private HashMap<String,Target> m_targets = new HashMap<>();
    
    public Limelight()
    {
        setLimelightPosition(0.0, 0.0, 0.0,
                             new AngleD(AngleD.DEGREES,0.0), new AngleD(AngleD.DEGREES,0.0));
    }
    
    public Limelight(double robotX, double robotY, double robotZ, AngleD heading, AngleD elevation)
    {
        setLimelightPosition(robotX, robotY, robotZ, heading, elevation);
    }
    
    /**
     * Set the position of the limelight on the robot.
     */
    public void setLimelightPosition(double robotX, double robotY, double robotZ,
        AngleD heading, AngleD elevation)
    {
        m_robotX = robotX;
        m_robotY = robotY;
        m_robotZ = robotZ;
        m_heading.setValue(heading);
        m_elevation.setValue(elevation);
        // build the transform that positions the limelight relative to the robot
        // by initializing the transform to an identify, applying a scaling to map
        // from a left handed coordinate system to a right handed, a rotation to
        // set the elevation angle, and a translation to set position of the limelight
        // on the robot.
        m_limelightToRobotXfm.identity();
        // rescale the axis system from a left handed to right handed axis system
        m_limelightToRobotXfm.scale(1.0, 1.0, -1.0);
        // rotate around X until the Y is straight up (90deg) plus the tilt angle for the
        // camera facing forward
        AngleD xRotation = new AngleD(AngleD.DEGREES, 90.0).add(m_elevation);
        m_limelightToRobotXfm.rotate(Xfm4x4d.AXIS_X, xRotation);
        // The turntable rotation in robot Z
        m_limelightToRobotXfm.rotate(Xfm4x4d.AXIS_Z, new AngleD(heading).mult(-1.0));
        // And finally the translation that places the limelight on the robot.
        m_limelightToRobotXfm.translate(m_robotX, m_robotY, m_robotZ);
        // invert that to buld the transform that describes where the robot
        // elements are relative to the limelight
        m_limelightToRobotXfm.invert(m_robotToLimelightXfm);
    }
    
    public Point3d getLocationOnRobot() {
        return new Point3d(m_robotX, m_robotY, m_robotZ);
    }
    
    /**
     *
     */
    public int addTarget(String identifier, Point3d centroid, Vector3d normal,
                          double width, double height)
    {
        m_targets.put(identifier, new Target(identifier, centroid, normal, width, height));
        return m_targets.size() - 1;
    }
    
    /**
     * Transform a robot-relative point to limelight-relative
     * point. This takes a robot-relative point, and
     * creates a limelight-relative point. The robot-relative
     * point is unchanged.
     */
    public Point3d xfmRobotPtToLimelight(Point3d robotPt)
    {
        return m_robotToLimelightXfm.transform(robotPt, new Point3d());
    }
    
    /**
     * Transform an array of robot-relative points to limelight-relative
     * points. This takes an array of robot-relative points, and
     * creates an array of limelight-relative points. The robot-relative
     * points are unchanged.
     */
    public Point3d[] xfmRobotPtsToLimelight(Point3d[] robotPts)
    {
        return m_robotToLimelightXfm.cloneAndTransform(robotPts);
    }
    
    /**
     * Transform a robot-relative vector to limelight-relative
     * vector. This takes a robot-relative vector, and
     * creates a limelight-relative vector. The robot-relative
     * vector is unchanged.
     */
    public Vector3d xfmRobotVecToLimelight(Vector3d robotV)
    {
        return m_robotToLimelightXfm.transform(robotV, new Vector3d());
    }
    
    /**
     * Transform an array of robot-relative vectors to limelight-relative
     * vectors. This takes an array of robot-relative vectors, and
     * creates an array of limelight-relative vectors. The robot-relative
     * vectors are unchanged.
     */
    public Vector3d[] xfmRobotVecsToLimelight(Vector3d[] robotVs)
    {
        return m_robotToLimelightXfm.cloneAndTransform(robotVs);
    }
    
    /**
     * Transform a limelight-relative point to robot-relative point and
     * return that robot-relative point. This is useful if you want to
     * display the limelight on a diagram of the robot.
     */
    public Point3d xfmLimelightPtToRobot(Point3d limelightPt)
    {
        return m_limelightToRobotXfm.transform(limelightPt, new Point3d());
    }
    
    /**
     * Transform the limelight-relative point geometry to robot-relative points
     * and return those robot-relative points. This is useful if you want to
     * display the limelight on a diagram of the robot.
     */
    public Point3d[] xfmLimelightPtGeoToRobot()
    {
        return m_limelightToRobotXfm.cloneAndTransform(LIMELIGHT_PT_GEOMETRY);
    }
    
    /**
     * Transform a limelight-relative vector to robot-relative vector and
     * return the robot-relative vector. This is useful if you want to
     * display the limelight on a diagram of the robot.
     */
    public Vector3d xfmLimelightVecToRobot(Vector3d limelightV)
    {
        return m_limelightToRobotXfm.transform(limelightV, new Vector3d());
    }
    
    /**
     * Transform the limelight-relative vector geometry to robot-relative vectors
     * and returns those robot-relative vectors. This is useful if you want to
     * display the limelight on a diagram of the robot.
     */
    public Vector3d[] xfmLimelightVecGeoToRobot()
    {
        return m_limelightToRobotXfm.cloneAndTransform(LIMELIGHT_V_GEOMETRY);
    }
    
    /**
     * Print an annotated list of the limelight geometry transformed to robot-relative
     * coordinates for the current camera position. This is primarily a debugging method.
     */
    public void printLimelightGeomToRobot()
    {
        printPoints(LIMELIGHT_PT_NAMES, xfmLimelightPtGeoToRobot());
        printVectors(LIMELIGHT_V_NAMES, xfmLimelightVecGeoToRobot());
    }
    
    /**
     * Print an annotated list of the limelight targets on the field. These Targets
     * are defined in field coordinates.
     */
    public void printLimelightTargets() {
        for (Target target: m_targets.values()) {
            target.printTarget();
        }
    }
    public AngleD[] getLimelightAnglesToTarget(Point3d targetLocation) {
        AngleD[] targetAngles = {
                new AngleD().atan2(targetLocation.getX(),targetLocation.getZ()),
                new AngleD().atan2(targetLocation.getY(),targetLocation.getZ())};
        return targetAngles;
    }

    /**
     *  This telemetry class does not monitor what the limelight sees, it simply uses the limelight
     *  reported sightings to compute the target position with respect to the robot. This method
     *  takes the target identifier for the target that is seen (other code identifies which is the
     *  most likely target) and computes the relationship between the target and the robot.
     *
     * @param targetIdentifier
     * @param limelightX
     * @param limelightY
     */
    public TargetPosition getTargetRelativeToRobot(String targetIdentifier,
                                                   AngleD limelightX, AngleD limelightY) {
        Target target = m_targets.get(targetIdentifier);
        if (null == target) {
            // An invalid target identifier was found.
            return null;
        }
        Vector3d limelightToTarget = new Vector3d(limelightX.tan(), limelightY.tan(), 1.0).normalize();
        Vector3d robotToTarget = xfmLimelightVecToRobot(limelightToTarget);
        AngleD heading = new AngleD().atan2(robotToTarget.getI(),robotToTarget.getJ());
        AngleD elevation = new AngleD().asin(robotToTarget.getK());
        double targetDistance = (target.m_centroid.getZ() - m_robotZ) / elevation.sin();
        Vector3d targetToLimelight = robotToTarget.cloneVector3d().reverse();
        Point3d limelightCenter = new Line3d(target.m_centroid,
                targetToLimelight).pointAtDistance(targetDistance);

        return new TargetPosition(targetIdentifier, robotToTarget, heading,
                                  elevation, targetDistance, limelightCenter);
    }
}


So lets test adding a target. To really do this we need some extra geometry for the limelight tape around the
hex target so I have added that to the field geometry. Additionally, we will add the opposing team's hex target to the field geometry - this has been done in the initial field geometry cell.

In [46]:
Limelight limelight = new Limelight();
limelight.addTarget("my hex goal", FIELD_POINTS[HEX_GOAL_TAPE_CENTROID], new Vector3d(0.0, -1.0, 0.0),
        FIELD_POINTS[HEX_GOAL_LL_TAPE_LR].getX()-FIELD_POINTS[HEX_GOAL_LL_TAPE_UL].getX(),
        FIELD_POINTS[HEX_GOAL_LL_TAPE_UL].getZ()-FIELD_POINTS[HEX_GOAL_LL_TAPE_LR].getZ());
limelight.addTarget("opposing hex goal", FIELD_POINTS[HEX_OP_GOAL_TAPE_CENTROID], new Vector3d(0.0, 1.0, 0.0),
        FIELD_POINTS[HEX_OP_GOAL_LL_TAPE_UL].getX()-FIELD_POINTS[HEX_OP_GOAL_LL_TAPE_LR].getX(),
        FIELD_POINTS[HEX_OP_GOAL_LL_TAPE_UL].getZ()-FIELD_POINTS[HEX_OP_GOAL_LL_TAPE_LR].getZ());
limelight.printLimelightTargets();


------ target 'my hex goal' ------
  centroid: (  1.709,  7.990,  2.284)
  normal:   (  0.000, -1.000,  0.000)
  width:      0.977
  height:     0.438
------ target 'opposing hex goal' ------
  centroid: ( -1.709, -8.730,  2.285)
  normal:   (  0.000,  1.000,  0.000)
  width:      0.977
  height:     0.438


And lets create a class that represents our robot, we will use that to keep track of robot geometry, camera geometry on the robot, and robot position on the field. For the robot geometry, we will be a really skinny robot so our orientation is really obvious in the transformations:

In [53]:
import org.a05annex.util.AngleD;
import org.a05annex.util.geo3d.Point3d;
import org.a05annex.util.geo3d.Xfm4x4d;

class MyRobot {
    
    // The robot coordinates
    static final int CENTER = 0;
    static final int RF = 1;
    static final int LF = 2;
    static final int LR = 3;
    static final int RR = 4;
    static final int LIMELIGHT = 5;

    /* The Z position (relative to the robot) of the limelight lens in meters. */
    static final double LIMELIGHT_HEIGHT = .52;
    /* The elevation angle of the camera relative to the XY plane (the floor) in radians. */
    static final double LIMELIGHT_ELEV_ANGLE = 0.337;
    /* The X position (relative to the robot) of the limelight lens in meters. */
    static final double LIMELIGHT_X_OFFSET = -0.20;
    /* The Y position (relative to the robot) of the limelight lens in meters. */
    static final double LIMELIGHT_Y_OFFSET = 0.14;
    
    static final String[] GEOMETRY_NAMES = {"center     : ", "right front: ", "left front : ",
                                            "left rear  : ", "right rear : ", "limelight  : " };
    static final Point3d[] ROBOT_GEOMETRY = {new Point3d(0.0, 0.0, 0.0), new Point3d(0.1, 0.3, 0.0),
        new Point3d(-0.1, 0.3, 0.0), new Point3d(-0.1, -0.3, 0.0), new Point3d(0.1, -0.3, 0.0),
        new Point3d(LIMELIGHT_X_OFFSET, LIMELIGHT_Y_OFFSET, LIMELIGHT_HEIGHT)};
    
    double m_fieldX = 0.0;
    double m_fieldY = 0.0;
    final AngleD m_heading = new AngleD(AngleD.RADIANS, 0.0);
    
    final Xfm4x4d m_robotToFieldXfm = new Xfm4x4d();
    final Xfm4x4d m_fieldToRobotXfm = new Xfm4x4d();
    
    final Limelight m_limelight = new Limelight();
    
    public MyRobot()
    {
    }
    
    public MyRobot(double fieldX, double fieldY, AngleD heading)
    {
        setFieldPosition(fieldX, fieldY, heading);
    }
    
    /**
     * Set the position of the robot on the field.
     */
    public void setFieldPosition(double fieldX, double fieldY, AngleD heading)
    {
        m_fieldX = fieldX;
        m_fieldY = fieldY;
        m_heading.setValue(heading);
        // build the transform that positions the robot relative to the field
        // by initializing the transform to an identify, applying the heading
        // rotation, and then translating the robot to some position on the field.
        m_robotToFieldXfm.identity();
        m_robotToFieldXfm.rotate(Xfm4x4d.AXIS_Z, heading.cloneAngleD().mult(-1.0));
        m_robotToFieldXfm.translate(fieldX, fieldY, 0.0);
        // invert that to buld the transform that describes where the field
        // elements are relative to the robot
        m_robotToFieldXfm.invert(m_fieldToRobotXfm);
    }
    
    public void setLimelightPosition(double robotX, double robotY, double robotZ,
                                     AngleD heading, AngleD elevation)
    {
        m_limelight.setLimelightPosition(robotX, robotY, robotZ, heading, elevation);
    }
    
    /**
     *
     */
    public int addLimelightTarget(String identifier, Point3d centroid, Vector3d normal,
                          double width, double height)
    {
        return m_limelight.addTarget(identifier,  centroid, normal, width, height);
    }
    /**
     * Transform a field-relative point to robot-relative
     * point. This takes afield-relative point, and
     * creates a robot-relative point. The field-relative
     * point is unchanged.
     */
    public Point3d xfmFieldPtToRobot(Point3d fieldPt)
    {
        return m_fieldToRobotXfm.transform(fieldPt, new Point3d());
    }
    
    /**
     * Transform a field-relative array of points to robot-relative
     * array of points. This takes an array of field-relative points, and
     * creates an array of robot-relative points. The field-relative
     * points are unchanged.
     */
    public Point3d[] xfmFieldPtsToRobot(Point3d[] fieldPts)
    {
        return m_fieldToRobotXfm.cloneAndTransform(fieldPts);
    }
    
    /**
     * Transform the a robot-relative point to a field-relative point and
     * return the field-relative point. This is useful if you want to
     * display the robot on a diagram of the field.
     */
    public Vector3d xfmRobotVecToField(Vector3d robotVec)
    {
        return m_robotToFieldXfm.transform(robotVec, new Vector3d());
    }
    
    /**
     * Transform the a robot-relative point to a field-relative point and
     * return the field-relative point. This is useful if you want to
     * display the robot on a diagram of the field.
     */
    public Point3d xfmRobotPtToField(Point3d robotPt)
    {
        return m_robotToFieldXfm.transform(robotPt, new Point3d());
    }
    
    /**
     * Transform the robot-relative geometry to field-relative coordinates and
     * return those field-relative coordinates. This is useful if you want to
     * display the robot on a diagram of the field.
     */
    public Point3d[] xfmRobotPtsToField()
    {
        return m_robotToFieldXfm.cloneAndTransform(ROBOT_GEOMETRY);
    }
    
    /**
     * Print an annotatted list of the robot geometry transformed to field-relative
     * coordinates for the current field position. This is primarily a debugging method.
     */
    public void printRobotPtsToField()
    {
        Point3d[] pts = xfmRobotPtsToField();
        for (int i = 0; i < pts.length; i++) {
            System.out.println(String.format("%s (%9.3f,%9.3f,%9.3f)", GEOMETRY_NAMES[i],
                                             pts[i].getX(), pts[i].getY(), pts[i].getZ()));
        }
    }
}

// MyRobot myRobot = new MyRobot();

// System.out.println("-- Raw robot coordinates when the robot is at center field --");
// myRobot.printRobotPtsToField();
// System.out.println("\n-- Robot field coordinates when at center field rotated 90 degrees --");
// myRobot.setFieldPosition(0.0, 0.0, new AngleD(AngleD.DEGREES,90.0));
// myRobot.printRobotPtsToField();

// System.out.println("\n-- Robot field coordinates when centered on the center of the start line --");
// myRobot.setFieldPosition(FIELD_POINTS[START_CENTER].getX(), FIELD_POINTS[START_CENTER].getY(), new AngleD(AngleD.RADIANS,0.0));
// myRobot.printRobotPtsToField();
// System.out.println("\n-- Robot centered on the center of the start line, heading 30 degrees --");
// System.out.println("-- Field elements --");
// printFieldPts(FIELD_POINTS);
// System.out.println("-- Robot points relative to the field --");
// myRobot.setFieldPosition(FIELD_POINTS[START_CENTER].getX(), FIELD_POINTS[START_CENTER].getY(), new AngleD(AngleD.DEGREES,30.0));
// myRobot.printRobotPtsToField();
// System.out.println("-- Field elements relative to the robot --");
// printFieldPts(myRobot.xfmFieldPtsToRobot(FIELD_POINTS));
//System.out.println("\n-- Robot field coordinates when centered on the center of the start line, heading 90 degrees --");
//myRobot.setFieldPosition(FIELD_POINTS[START_CENTER].getX(), FIELD_POINTS[START_CENTER].getY(), new AngleD(AngleD.DEGREES,90.0));
//myRobot.printRobotPtsToField();
//System.out.println("-- Field elements relative to the robot --");
//printFieldPts(myRobot.xfmFieldPtsToRobot(FIELD_POINTS));


In [76]:
double xCenter = FIELD_POINTS[HEX_GOAL_TAPE_CENTROID].getX();
double yCenter = FIELD_POINTS[HEX_GOAL_TAPE_CENTROID].getY() - 5.0379;
double zCenter = 0.520;

MyRobot myRobot = new MyRobot();
myRobot.setLimelightPosition(1.0,0.0,0.520,
    new AngleD(AngleD.DEGREES,0.0), new AngleD(AngleD.RADIANS,0.337));
myRobot.addLimelightTarget("my hex goal", FIELD_POINTS[HEX_GOAL_TAPE_CENTROID], new Vector3d(0.0, -1.0, 0.0),
        FIELD_POINTS[HEX_GOAL_LL_TAPE_LR].getX()-FIELD_POINTS[HEX_GOAL_LL_TAPE_UL].getX(),
        FIELD_POINTS[HEX_GOAL_LL_TAPE_UL].getZ()-FIELD_POINTS[HEX_GOAL_LL_TAPE_LR].getZ());
myRobot.addLimelightTarget("opposing hex goal", FIELD_POINTS[HEX_OP_GOAL_TAPE_CENTROID], new Vector3d(0.0, 1.0, 0.0),
        FIELD_POINTS[HEX_OP_GOAL_LL_TAPE_UL].getX()-FIELD_POINTS[HEX_OP_GOAL_LL_TAPE_LR].getX(),
        FIELD_POINTS[HEX_OP_GOAL_LL_TAPE_UL].getZ()-FIELD_POINTS[HEX_OP_GOAL_LL_TAPE_LR].getZ());
myRobot.m_limelight.printLimelightTargets();


Limelight.TargetPosition targetPosition = myRobot.m_limelight.getTargetRelativeToRobot(
    "my hex goal", new AngleD(AngleD.DEGREES,0.0), new AngleD(AngleD.DEGREES,0.0));
targetPosition.printTargetPosition();

// robot heading as specified for testing ...
myRobot.setFieldPosition(0.0, 0.0, new AngleD(AngleD.DEGREES,90.0));
Vector3d fieldToTarget = myRobot.xfmRobotVecToField(targetPosition.robotToTarget);
printVector("field to target:", fieldToTarget);
Vector3d targetToField = fieldToTarget.cloneVector3d().reverse();
printVector("target to field:", targetToField);
Point3d limelightCenter = new Line3d(FIELD_POINTS[HEX_GOAL_TAPE_CENTROID],
        targetToField).pointAtDistance(targetPosition.targetDistance);
printPoint("field limelight center:", limelightCenter);
Vector3d limelightCtrToRobotCtr =
    new Vector3d(myRobot.m_limelight.getLocationOnRobot(), new Point3d(0.0,0.0,0.0));
printVector("limelight to robot:", limelightCtrToRobotCtr);
Vector3d fieldLimelightToRobot = myRobot.xfmRobotVecToField(limelightCtrToRobotCtr);
printVector("field limelight to robot::", fieldLimelightToRobot);
Point3d robotCenter = new Line3d(limelightCenter,
        fieldLimelightToRobot).pointAtDistance(1.0);
printPoint("field robot center:", robotCenter);



------ target 'my hex goal' ------
  centroid: (  1.709,  7.990,  2.284)
  normal:   (  0.000, -1.000,  0.000)
  width:      0.977
  height:     0.438
------ target 'opposing hex goal' ------
  centroid: ( -1.709, -8.730,  2.285)
  normal:   (  0.000,  1.000,  0.000)
  width:      0.977
  height:     0.438
------ target position for 'my hex goal' ------
  robot to target: (  0.000,  0.944,  0.331)
  target heading:      0.000
  target elevation:   19.309
  target_distance:     5.335
field to target: (  0.944,  0.000,  0.331)
target to field: ( -0.944, -0.000, -0.331)
field limelight center: ( -3.327,  7.990,  0.520)
limelight to robot: ( -1.000,  0.000, -0.520)
field limelight to robot:: ( -0.000,  1.000, -0.520)
field robot center: ( -3.327,  8.990,  0.000)


When we review this we see that the field position after any translation and/or rotation is what we expected with
some round-off error. Note that when the robot is centered on the start line and turns to a heading of 30&deg;
the robot is almost directly facing the shooting goal (the X offset in robot coordinates is almost 0.0).

OK, this is pretty cool, but how do we work the limelight into this? Well, lets create a `Limelight` class that
represents the limelight relative to the robot so we can build the transformations between limelight coordinates
and robot coordinates. The `Limelight` class will look almost like the `MyRobot` class except that it will be
expressing the relationship between the limelight and the robot instead of the robot and the field:

In [37]:
import org.a05annex.util.AngleD;
import org.a05annex.util.geo3d.Point3d;
import org.a05annex.util.geo3d.Xfm4x4d;

class Limelight {
    
    // The limelight coordinates
    static final int CENTER = 0;
    static final int UNIT_X = 1;
    static final int UNIT_Y = 2;
    static final int UNIT_Z = 3;

    static final String[] LIMELIGHT_NAMES = {"center : ",
                                             "unit X : ",
                                             "unit Y : ",
                                             "unit Z : "};
    static final Point3d[] LIMELIGHT_GEOMETRY = {new Point3d(0.0, 0.0, 0.0),
        new Point3d(1.0, 0.0, 0.0), new Point3d(0.0, 1.0, 0.0), new Point3d(0.0, 0.0, 1.0)};
    
    double m_robotX = 0.0;
    double m_robotY = 0.0;
    double m_robotZ = 0.0;
    final AngleD m_elevation = new AngleD(AngleD.RADIANS, 0.0);
    
    Xfm4x4d m_limelightToRobotXfm = new Xfm4x4d();
    Xfm4x4d m_robotToLimelightXfm = new Xfm4x4d();
    
    public Limelight()
    {
    }
    
    public Limelight(double robotX, double robotY, double robotZ, AngleD elevation)
    {
        setLimelightPosition(robotX, robotY, robotZ, elevation);
    }
    
    /**
     * Set the position of the limelight on the robot.
     */
    public void setLimelightPosition(double robotX, double robotY, double robotZ, AngleD elevation)
    {
        m_robotX = robotX;
        m_robotY = robotY;
        m_robotZ = robotZ;
        m_elevation.setValue(elevation);
        // build the transform that positions the limelight relative to the robot
        // by initializing the transform to an identify, applying a scaling to map
        // from a left handed coordinate system to a right handed, a rotation to
        // set the elevation angle, and a translation to set position of the limelight
        // on the robot.
        m_limelightToRobotXfm.identity();
        // rescale the axis system from a left handed to right handed axis system
        m_limelightToRobotXfm.scale(1.0, 1.0, -1.0);
        // rotate around X until the Y is straight up (90deg) plus the tilt angle for the
        // camera facing forward
        AngleD xRotation = new AngleD(AngleD.DEGREES, 90.0).add(m_elevation);
        m_limelightToRobotXfm.rotate(Xfm4x4d.AXIS_X, xRotation);
        m_limelightToRobotXfm.translate(m_robotX, m_robotY, m_robotZ);
        // invert that to buld the transform that describes where the robot
        // elements are relative to the limelight
        m_limelightToRobotXfm.invert(m_robotToLimelightXfm);
    }
    
    /**
     * Transform a robot-relative point to limelight-relative
     * point. This takes an a robot-relative point, and
     * creates a limelight-relative point. The robot-relative
     * point in unchanged.
     */
    public Point3d xfmRobotPtToLimelight(Point3d robotPt)
    {
        return m_robotToLimelightXfm.transform(robotPt, new Point3d());
    }
    
    /**
     * Transform an array of robot-relative coordinates to limelight-relative
     * coordinates. This takes an array of robot-relative coordinates, and
     * creates an array of limelight-relative coordinates. The robot-relative
     * coordinates are unchanged.
     */
    public Point3d[] xfmRobotPtsToLimelight(Point3d[] robotPts)
    {
        Point3d[] limelightPts = new Point3d[robotPts.length];
        for (int i = 0; i < robotPts.length; i++) {
            limelightPts[i] = robotPts[i].clonePoint3d();
        }
        return m_robotToLimelightXfm.transform(limelightPts);
    }
    
    /**
     * Transform the limelight-relative geometry to robot-relative coordinates and
     * return those robot-relative coordinates. This is useful if you want to
     * display the limelight on a diagram of the robot.
     */
    public Point3d[] xfmLimelightPtsToRobot()
    {
        Point3d[] robotPts = new Point3d[LIMELIGHT_GEOMETRY.length];
        for (int i = 0; i < LIMELIGHT_GEOMETRY.length; i++) {
            robotPts[i] = LIMELIGHT_GEOMETRY[i].clonePoint3d();
        }
        return m_limelightToRobotXfm.transform(robotPts);
    }
    
    /**
     * Print an annotatted list of the limelight geometry transformed to robot-relative
     * coordinates for the current camera position. This is primarily a debugging method.
     */
    public void printLimelightPtsToRobot()
    {
        Point3d[] pts = xfmLimelightPtsToRobot();
        for (int i = 0; i < pts.length; i++) {
            System.out.println(String.format("%s (%7.3f,%7.3f,%7.3f)", LIMELIGHT_NAMES[i],
                                             pts[i].getX(), pts[i].getY(), pts[i].getZ()));
        }
    }
}

Limelight myLimelight = new Limelight();
//myLimelight.setLimelightPosition(0.0,0.0,0.0,new AngleD(AngleD.DEGREES,0.0));
//System.out.println("-- Robot coordinates of limelight at (0,0,0), elevation = 0");
//myLimelight.printLimelightPtsToRobot();

//myLimelight.setLimelightPosition(-0.200, 0.140, 0.520, new AngleD(AngleD.DEGREES,0.0));
//System.out.println("\n-- Robot coordinates of limelight at (-0.200,0.140,0.520), elevation = 0");
//myLimelight.printLimelightPtsToRobot();

myLimelight.setLimelightPosition(-0.200, 0.140, 0.520, new AngleD(AngleD.RADIANS,0.337));
//System.out.println("\n-- Robot coordinates of limelight at (-0.200,0.140,0.520), elevation = 0.337rad");
//myLimelight.printLimelightPtsToRobot();

System.out.println("\n-- Robot field coordinates when centered on the center of the start line --");
System.out.println("-- field locations --");
printFieldPts(FIELD_POINTS);
myRobot.setFieldPosition(FIELD_POINTS[START_CENTER].getX(), FIELD_POINTS[START_CENTER].getY(), new AngleD(AngleD.DEGREES,30.0));
System.out.println("-- Robot points relative to the field --");
myRobot.printRobotPtsToField();
System.out.println("-- Field elements relative to the robot --");
printFieldPts(myRobot.xfmFieldPtsToRobot(FIELD_POINTS));
System.out.println("-- Field elements relative to the limelight --");
printFieldPts(myLimelight.xfmRobotPtsToLimelight(myRobot.xfmFieldPtsToRobot(FIELD_POINTS)));





-- Robot field coordinates when centered on the center of the start line --
-- field locations --
field center     :  (    0.000,    0.000,    0.000)
start center     :  (    0.000,    5.000,    0.000)
start right      :  (    1.709,    5.000,    0.000)
start left       :  (   -1.560,    5.000,    0.000)
trench corner    :  (    2.695,    2.743,    0.000)
hex goal center  :  (    1.709,    7.990,    2.500)
round goal center:  (    1.709,    8.730,    2.500)
-- Robot points relative to the field --
center     :  (    0.000,    5.000,    0.000)
right front:  (    0.237,    5.210,    0.000)
left front :  (    0.063,    5.310,    0.000)
left rear  :  (   -0.237,    4.790,    0.000)
right rear :  (   -0.063,    4.690,    0.000)
limelight  :  (   -0.103,    5.221,    0.520)
-- Field elements relative to the robot --
field center     :  (    2.500,   -4.330,    0.000)
start center     :  (    0.000,    0.000,    0.000)
start right      :  (    1.480,    0.854,    0.000)
start left       :  (

And, this last bit, `-- Field elements relative to the limelight --` is really interesting
to us, especially the `hex goal center`. If we imagine the limelight tape was a square at the center
of the hex target, then the point (0.185,0.776,3.773) is the position of the centroid of the limelight
target relative to the limelight camera lens (at (0,0,0)). So the limelight X angle is
atan2(0.185,3.773) and the Y angle is atan2(0.776,3.773):

In [51]:
AngleD limelightX = new AngleD().atan2(0.185,3.773);
System.out.println(String.format("limelight X angle %.2f degree (%.3f radians)",
    limelightX.getDegrees(), limelightX.getRadians()));
    
AngleD limelightY = new AngleD().atan2(0.776,3.773);
System.out.println(String.format("limelight Y angle %.2f degree (%.3f radians)",
    limelightY.getDegrees(), limelightY.getRadians()));

limelight X angle 2.81 degree (0.049 radians)
limelight Y angle 11.62 degree (0.203 radians)


OK, now we have the basic components of the math we need to compute where the robot is on the field. We will save working through the details of that for the next session.